### This time start without sampling and test with LSTM

Without sampling usually failed bz it is unbalanced class dataset and you need to oversampling and undersampling.

Decision Tree ( av-precison = 0.258728) without sampling  
DNN-LSTM ( av-precison = 0.25872) without sampling

In [1]:
import os 
import numpy as np 
import pandas as pd

In [2]:
train_feature = np.load('Data/mix/train_feature.npy')
valid_feature = np.load('Data/mix/valid_feature.npy')
test_feature = np.load('Data/mix/test_feature.npy')
train_label = np.load('Data/mix/train_label.npy')
valid_label = np.load('Data/mix/valid_label.npy')
test_label = np.load('Data/mix/test_label.npy')

In [3]:
train_feature.shape, train_label.shape, test_feature.shape, test_label.shape, valid_feature.shape, valid_label.shape

((12217414, 3),
 (12217414, 4),
 (3817942, 3),
 (3817942, 4),
 (3054354, 3),
 (3054354, 4))

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report

def eval_metrics(classifier, test_features, test_labels):
    
    # make prediction
    predictions   = classifier.predict(test_features)
    
    base_score   = classifier.score(test_features, test_labels)
    accuracy = accuracy_score(test_labels, predictions)
    av_precision = average_precision_score(test_labels, predictions)
    
    target_names = ['StartHesitation','Turn','Walking', 'All_zero']
    print("Classification report")
    print("---------------------","\n")
    print(classification_report(test_labels, predictions, target_names=target_names),"\n")

    print("Accuracy Measures")
    print("---------------------","\n")
    print("Base score: ", base_score)
    print("Accuracy: ", accuracy)
    print("Avarge Precision: ", av_precision)
    
    return base_score,accuracy,av_precision

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
Models = {                         
    "Decision Tree": DecisionTreeClassifier(),      
    "KNearest": KNeighborsClassifier(n_jobs=-1),           
    }

In [6]:
import mlflow
import mlflow.tensorflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [7]:
counter = 1
for Model_Name, classifier in Models.items(): 
    # with mlflow.start_run(nested=True):
    print(f"{counter}. {Model_Name}")
    
    with mlflow.start_run():
        # fit the model
        from joblib import parallel_backend
        with parallel_backend('threading', n_jobs=-1):
            classifier.fit(train_feature, train_label)
        counter = counter + 1
        
        # Calculate the metrics
        base_score,accuracy,av_precision = eval_metrics(classifier,
                                                        valid_feature,
                                                        valid_label)  
        
        mlflow.log_param("Model"           , Model_Name)
        mlflow.log_param("Dataset" , "Mix")
        mlflow.log_metric("base_score"     , base_score)
        mlflow.log_metric("accuracy"       , accuracy)
        mlflow.log_metric("av_precision"   , av_precision)
        
        if av_precision > 0.95 :
            mlflow.sklearn.log_model(classifier,Model_Name, signature=signature)
            print(f"f1 socre is more than 0.945 so the {Model_Name} is saved")
        else :
            print(f"Because f1 socre is not quality. The model is skip to saving phase.")
        
        print("________________________________________")

1. Decision Tree
Classification report
--------------------- 



/home/hanlinn/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                 precision    recall  f1-score   support

StartHesitation       0.12      0.13      0.13     48579
           Turn       0.32      0.33      0.33    358311
        Walking       0.07      0.08      0.08     48687
       All_zero       0.90      0.89      0.90   2598777

      micro avg       0.80      0.80      0.80   3054354
      macro avg       0.35      0.36      0.36   3054354
   weighted avg       0.81      0.80      0.80   3054354
    samples avg       0.80      0.80      0.80   3054354
 

Accuracy Measures
--------------------- 

Base score:  0.801690635728537
Accuracy:  0.801690635728537
Avarge Precision:  0.2823923637703956
Because f1 socre is not quality. The model is skip to saving phase.
________________________________________
2. KNearest
Classification report
--------------------- 

                 precision    recall  f1-score   support

StartHesitation       0.28      0.05      0.08     48579
           Turn       0.46      0.26      0.33    358311
   

/home/hanlinn/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Tensorflow

In [8]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from datetime import datetime

2023-05-09 18:18:57.789047: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
epoch = 5
input_shape = 3

inputs = layers.Input(shape=input_shape, name="input_layer")
x = layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(inputs)
x = layers.LSTM(64)(x) # return vector for whole sequence
x = layers.Dense(16, activation="relu")(x) 
outputs = layers.Dense(4, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs, name="model_LSTM")



model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model.summary()

2023-05-09 18:18:59.111452: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-09 18:18:59.130534: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-09 18:18:59.130746: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 3)]               0         
                                                                 
 lambda (Lambda)             (None, 3, 1)              0         
                                                                 
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dense (Dense)               (None, 16)                1040      
                                                                 
 dense_1 (Dense)             (None, 4)                 68        
                                                                 
Total params: 18,004
Trainable params: 18,004
Non-trainable params: 0
_________________________________________________________________


2023-05-09 18:19:00.044129: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-09 18:19:00.045383: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-09 18:19:00.046376: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [10]:
# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=5) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=3,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_feature, train_label))
train_dataset =  train_dataset.batch(2048).prefetch(tf.data.AUTOTUNE)

valid_dataset = tf.data.Dataset.from_tensor_slices((valid_feature, valid_label))
valid_dataset = valid_dataset.batch(2048).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_feature,test_label))
test_dataset = test_dataset.batch(2048).prefetch(tf.data.AUTOTUNE)

train_dataset, valid_dataset, test_dataset

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 4), dtype=tf.float64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 4), dtype=tf.float64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 4), dtype=tf.float64, name=None))>)

In [12]:
# fit the model
start = datetime.now()
with mlflow.start_run():
    mlflow.tensorflow.autolog()
    
    # fit the model
    # with tf.device('/GPU:0'):
    start = datetime.now()
    history_model = model.fit(train_dataset,
                            batch_size=2048,
                            steps_per_epoch=len(train_dataset),
                            validation_data=valid_dataset,
                            validation_steps=int(len(valid_dataset)),
                            callbacks=[early_stopping, reduce_lr],
                            epochs=epoch) 
    end = datetime.now()
    print(f"The time taken to train the model is {end - start}")
        
    # Evaluate model
    model.evaluate(test_dataset)
    
    # Calculate the metrics
    # model_results = eval_metrics(model, 
    #                              test_features = test_feature,
    #                              test_labels = test_label)
    
    mlflow.log_param("Model"   , "LSTM")
    mlflow.log_param("Dataset" , "Mix")
    # mlflow.log_params(model_results)
    mlflow.tensorflow.autolog()

print("________________________________________")

Epoch 1/5


2023-05-09 18:19:01.425024: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [12217414,4]
	 [[{{node Placeholder/_1}}]]
2023-05-09 18:19:01.641275: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-09 18:19:01.643024: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder ten

5963/5966 [============================>.] - ETA: 0s - loss: 0.4093 - accuracy: 0.8581

2023-05-09 18:19:22.923232: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [3054354,4]
	 [[{{node Placeholder/_1}}]]
2023-05-09 18:19:23.115894: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-09 18:19:23.117024: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tens

5966/5966 [==============================] - 26s 4ms/step - loss: 0.4093 - accuracy: 0.8581 - val_loss: 0.3899 - val_accuracy: 0.8620 - lr: 0.0010
Epoch 2/5
5966/5966 [==============================] - 22s 4ms/step - loss: 0.3870 - accuracy: 0.8631 - val_loss: 0.3865 - val_accuracy: 0.8624 - lr: 0.0010
Epoch 3/5
5966/5966 [==============================] - 22s 4ms/step - loss: 0.3843 - accuracy: 0.8635 - val_loss: 0.3834 - val_accuracy: 0.8630 - lr: 0.0010
Epoch 4/5
5966/5966 [==============================] - 22s 4ms/step - loss: 0.3819 - accuracy: 0.8637 - val_loss: 0.3805 - val_accuracy: 0.8639 - lr: 0.0010
Epoch 5/5
5966/5966 [==============================] - 22s 4ms/step - loss: 0.3806 - accuracy: 0.8638 - val_loss: 0.3791 - val_accuracy: 0.8642 - lr: 0.0010


...

INFO:tensorflow:Assets written to: /tmp/tmpukvouatz/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpukvouatz/model/data/model/assets
2023/05/09 18:21:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


The time taken to train the model is 0:02:03.413489
   1/1865 [..............................] - ETA: 59s - loss: 0.3786 - accuracy: 0.8662

2023-05-09 18:21:04.528637: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [3817942,4]
	 [[{{node Placeholder/_1}}]]


1865/1865 [==============================] - 5s 3ms/step - loss: 0.3796 - accuracy: 0.8642
________________________________________


In [13]:

# Calculate the metrics
model_preds_probs   = model.predict(test_feature)
av_precision    = average_precision_score(test_label, model_preds_probs)
av_precision

119311/119311 [==============================] - 123s 1ms/step


0.42159217384679054

#Mix DNN model

In [14]:
# Build the model 
epoch = 10
input_shape = 3

inputs = layers.Input(shape=input_shape, name="input_layer")
x = layers.Dense(32, activation ='relu')(inputs)
x = layers.Dense(64, activation = 'relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.1)(x)
# x = layers.Dense(128, activation ='relu')(x)
# x = layers.Dense(512, activation = 'relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.25)(x)
# x = layers.Dense(1024, activation ='relu')(x)
# x = layers.Dense(512, activation = 'relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(128, activation="relu")(x)
# x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
outputs = layers.Dense(4, activation="softmax",name="output_layer")(x)      
model_DNN = tf.keras.Model(inputs, outputs) 

model_DNN.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model_DNN.summary()



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 3)]               0         
                                                                 
 dense_2 (Dense)             (None, 32)                128       
                                                                 
 dense_3 (Dense)             (None, 64)                2112      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 output_layer (Dense)        (None, 4)                 132       
                                                                 
Total params: 4,452
Trainable params: 4,452
Non-trainable params: 0
_________________________________________________________________


In [15]:
from sklearn.model_selection import cross_val_score
counter = 1
Model_Name = 'DNN_model'
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)
tf.set_seed = 42
np.random.seed(40)

with mlflow.start_run():
    mlflow.tensorflow.autolog()
    
    # fit the model
    # with tf.device('/GPU:0'):
    start = datetime.now()
    history_model = model_DNN.fit(train_dataset,
                                  batch_size=2048,
                                  steps_per_epoch=len(train_dataset),
                                  validation_data=valid_dataset,
                                  validation_steps=int(len(valid_dataset)),
                                  callbacks=[early_stopping, reduce_lr],
                                  epochs=epoch) 
    end = datetime.now()
    print(f"The time taken to train the model is {end - start}")
        
    # Evaluate model
    model.evaluate(test_dataset)
    
    # Calculate the metrics
    # model_results = eval_metrics(model, 
    #                              test_features = test_feature,
    #                              test_labels = test_label)
    
    mlflow.log_param("Model"           , Model_Name)
    mlflow.log_param("Dataset" , "Defog")
    # mlflow.log_params(model_results)
    mlflow.tensorflow.autolog()
    
    
    print("________________________________________")

Epoch 1/10
5966/5966 [==============================] - 21s 3ms/step - loss: 0.4037 - accuracy: 0.8581 - val_loss: 0.3902 - val_accuracy: 0.8600 - lr: 0.0010
Epoch 2/10
5966/5966 [==============================] - 20s 3ms/step - loss: 0.3890 - accuracy: 0.8611 - val_loss: 0.3855 - val_accuracy: 0.8624 - lr: 0.0010
Epoch 3/10
5966/5966 [==============================] - 20s 3ms/step - loss: 0.3854 - accuracy: 0.8624 - val_loss: 0.3836 - val_accuracy: 0.8632 - lr: 0.0010
Epoch 4/10
5966/5966 [==============================] - 20s 3ms/step - loss: 0.3836 - accuracy: 0.8629 - val_loss: 0.3817 - val_accuracy: 0.8633 - lr: 0.0010
Epoch 5/10
5966/5966 [==============================] - 20s 3ms/step - loss: 0.3822 - accuracy: 0.8632 - val_loss: 0.3808 - val_accuracy: 0.8637 - lr: 0.0010
Epoch 6/10
5966/5966 [==============================] - 20s 3ms/step - loss: 0.3812 - accuracy: 0.8634 - val_loss: 0.3817 - val_accuracy: 0.8624 - lr: 0.0010
Epoch 7/10
5966/5966 [==============================

2023-05-09 18:27:52.757868: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [12217414,4]
	 [[{{node Placeholder/_1}}]]


The time taken to train the model is 0:03:24.491315
1865/1865 [==============================] - 5s 3ms/step - loss: 0.3796 - accuracy: 0.8642
________________________________________


In [16]:
# Calculate the metrics
model_preds_probs   = model.predict(test_feature)
av_precision    = average_precision_score(test_label, model_preds_probs)
print(av_precision)

119311/119311 [==============================] - 124s 1ms/step


ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets